# LOAD DATA

**Objective**: Load and adapt the data provided and additional

Loaded data in the following order:
1. [Provided](#1-Provided-data)
    - [Corn.paquet](#1.1-Main-dataset---Corn.parquet)
    - [Corn futures](#1.2-Corn-Futures)
    - [Chemicals](#1.3-Chemicals)
    - [Climate](#1.4-Climate)
2. [Additional](#2.-Additional-datasets)
    - [Agricultural](#2.1-Agricultural-info-in-US)
    - [Cropland in US](#1.2-Cropland-use-for-cops-in-all-US)
    - [Cropland by state](#2.3-Cropland-use-for-cops-in-each-state)
    - [Ethanol produced in US](#2.4-Ethanol-produced-US)

## Load packages

In [ ]:
# Set Git path
import os 
CURRENT_PATH = os.getcwd()

# Data manipulation
import pandas as pd
import csv
import xlrd
import openpyxl
from datetime import datetime
from ydata_profiling import ProfileReport


In [ ]:
if not os.path.exists('../data/'):
    os.makedirs('../data/')

## 1. Provided data

### 1.1 Main dataset - Corn.parquet

In [ ]:
corn_raw = pd.read_parquet('../exdata/provided/CORN.parquet/part-00000-79520c00-c34f-45a5-abf4-58866e63cb2f-c000.snappy.parquet')
#corn_raw.head()

#### Check filters and get the main only with the output feature:

1.            FILTER the dataset for STATISTICCAT == Area Planted, Acres Harvested, Yield

2.            Filter for AGG_LEVEL_DESC == STATE

3.            SHORT_DESC == CORN - ACRES PLANTED

4.            REFERENCE_PERIOD_DESC == YEAR

All filters applied

In [ ]:
# Apply filters
filter1 = corn_raw[corn_raw['STATISTICCAT_DESC'].isin(['AREA PLANTED','AREA HARVESTED','YIELD'])]
filter2 = filter1[filter1['AGG_LEVEL_DESC'] == 'STATE']
filter3 = filter2[filter2['SHORT_DESC'] == 'CORN - ACRES PLANTED']
filter4 = filter3[filter3['REFERENCE_PERIOD_DESC'] == 'YEAR']
#filter4.head()

To check if any of the other columns apart from: **'VALUE'**,**'STATISTICCAT_DESC'**, **'LOCATION_DESC'** and **'YEAR'** are relevant

In [ ]:
profile_filter4 = ProfileReport(filter4)
profile_filter4

In [ ]:
# Get main DF from filters
maindf = filter4[['VALUE','STATISTICCAT_DESC', 'LOCATION_DESC', 'YEAR']].copy()

profile_maindf = ProfileReport(maindf)
profile_maindf

After checking the profiling, there are needed changes in the **VALUE** and **LOCATION_DESC** columns

In [ ]:
# Give correct format to value column
maindf['VALUE'] = maindf['VALUE'].str.replace(r',[^,]*$', '', regex=True)
maindf['VALUE'] = maindf['VALUE'].str.replace(',', '').astype(float)

# Only rows with LOCATION_DESC defined
maindf = maindf[maindf['LOCATION_DESC'] != 'OTHER STATES']

In [ ]:
# Pivot to use each STATISTICCAT_DESC as feature
main_output = maindf.pivot(index=['YEAR','LOCATION_DESC'], columns=['STATISTICCAT_DESC'], values='VALUE')
main_output.reset_index(inplace=True)

In [ ]:
main_output

#### Main with more features besides filters

Applied:
- FILTER 2: **AGG_LEVEL_DESC** == *STATE*
- FILTER 4: **REFERENCE_PERIOD_DESC** == *YEAR*
- And take all combinations of **STATISTICCAT_DESC** and **SHORT_DESC** features

In [ ]:
filter2 = corn_raw[corn_raw['AGG_LEVEL_DESC'] == 'STATE']
filter4 = filter2[filter2['REFERENCE_PERIOD_DESC'] == 'YEAR']
main = filter4.copy()

# remain with the same subset that main
main = main[['VALUE','STATISTICCAT_DESC', 'SHORT_DESC', 'LOCATION_DESC', 'YEAR']].copy()

In [ ]:
# Give correct format to value column
main['VALUE'] = main['VALUE'].str.replace(r',[^,]*$', '', regex=True)
main['VALUE'] = main['VALUE'].str.replace(',', '')
main['VALUE'] = pd.to_numeric(main['VALUE'], errors='coerce')

# Only rows with LOCATION_DESC defined
main = main[main['LOCATION_DESC'] != 'OTHER STATES']

In [ ]:
# Pivot to use each SHORT_DESC as feature
main_pivot = main.pivot_table(index=['YEAR','LOCATION_DESC'], columns=['SHORT_DESC'], values='VALUE')

# Filter by y data -> CORN - ACRES PLANTED
main_pivot.dropna(subset=['CORN - ACRES PLANTED'], inplace=True)

main_pivot.reset_index(inplace=True)
main_pivot = main_pivot.rename_axis(None, axis=1)

main_pivot

#### Save main datasets

In [ ]:
main_output.to_pickle('../data/main_output.pkl')
main_pivot.to_pickle('../data/main.pkl')

### 1.2 Corn Futures

In [ ]:
futures_raw = pd.read_csv('../exdata/provided/Corn_Futures.csv')

# save only values for mean calculated from HIGH and LOW
futures_raw = futures_raw.assign(Futures = (futures_raw['High'] + futures_raw['Low'])/2)

# Rename column date
futures = futures_raw[['Date', 'Futures']].copy()
rename_cols = {'Date': 'DATE'}
futures.rename(columns=rename_cols, inplace=True)

# Format date format equally than in main df
futures['DATE'] = pd.to_datetime(futures['DATE'], format='%m/%d/%Y')
futures['DATE'] = futures['DATE'].dt.strftime('%Y-%m-%d')

# Ver correlación de media/mediana/sd con estado por año
futures


In [ ]:
futures.to_pickle('../data/futures.pkl')

### 1.3 Chemicals

In [ ]:
chem = pd.read_excel('../exdata/provided/WPU0652013A.xls', index_col=None, na_values=['NA'], usecols="A,B", skiprows=10)

# Rename column date
rename_cols = {'observation_date': 'DATE', 'WPU0652013A': 'chem'}
chem.rename(columns=rename_cols, inplace=True)

# Ver correlación de media/mediana/sd con estado por año
chem

In [ ]:
chem.to_pickle('../data/chem.pkl')

### 1.4 Climate

In [ ]:
# Create dictionary with all subfolders and pathfiles

pathfiles_dict = {}
files_dict = {}

for folder_path, folders, files in os.walk('../exdata/provided/Climate/'):
    # Create a list to store filenames for the current subfolder
    subfolder_filenames = []
    for file in files:
        # Append the filename to the list
        subfolder_filenames.append(os.path.join(folder_path, file))
    # Store the list of filenames in the dictionary with the subfolder path as the key
    pathfiles_dict[folder_path] = subfolder_filenames

pathfiles_dict.pop('../exdata/provided/Climate/')

prefix_to_remove = '../exdata/provided/Climate/'
for key in pathfiles_dict:
    updated_key = key.replace(prefix_to_remove, '', 1)
    files_dict[updated_key] = pathfiles_dict[key]

In [ ]:
# Get dataframe of all metrics and pivot it

metric = pd.DataFrame()

def get_start_row(file):
    with open(file, 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            # Check if the row matches the condition
            if row[0] == 'Date':
                start_row = i
                break
    return start_row

for key in files_dict.keys():
    for file in files_dict[key]:
        if file.endswith('.csv'):  # Check if the file is a CSV file
            # Read state and metric info
            df = pd.read_csv(file, nrows=1, header=None)
            # Find in which row starts the values
            start_row = get_start_row(file)
            # Append the data to the main dataframe
            temp_df = pd.read_csv(file, skiprows = start_row, header=0)
            temp_df = temp_df.assign(State = df[0][0], Metric = df[1][0])
            metric = metric.append(temp_df, ignore_index=True)


# remove anomaly column and rename Date
metric = metric.drop('Anomaly', axis=1)
rename_cols = {'Date': 'DATE'}
metric.rename(columns=rename_cols, inplace=True)

# pivot to get climate df
climate = metric.pivot(index=['DATE','State'], columns='Metric', values='Value')
climate.reset_index(inplace=True)

In [ ]:
# Format date format equally than in main df  - here only year and month
climate['DATE'] = pd.to_datetime(climate['DATE'], format='%Y%m')
climate['DATE'] = climate['DATE'].dt.strftime('%Y-%m')

# Ver correlación de media/mediana/sd con estado por año
climate = climate.rename_axis(None, axis=1)
climate

In [ ]:
climate.to_pickle('../data/climate.pkl')

## 2. Additional datasets

Links with info about this data can be found in:
- Agricultural info in US:
- Cropland in US and by state:
- Ethanol consumed: 

### 2.1 Agricultural info in US

In [ ]:
# Read exdata
agriculture = pd.read_excel('../exdata/additional/table01.xlsx', index_col=None, skiprows=2)
agriculture = agriculture.iloc[:72]

# Adapt colname YEAR
rename_cols = {'Year': 'YEAR'}
agriculture.rename(columns=rename_cols, inplace=True)

agriculture.to_pickle('../data/agriculture.pkl')
agriculture

### 2.2 Cropland use for cops in all US 

In [ ]:
# Read exdata
cropland_us = pd.read_excel('../exdata/additional/summary_Table_3_cropland_used_for_crops_19102022_update.xlsx', skiprows=1)
cropland_us = cropland_us.iloc[:113]

# Fill nas
cropland_us = cropland_us.fillna(cropland_us.median())

# Adapt colname YEAR
cropland_us.at[cropland_us.index[-1], 'Year 1/'] = 2022
rename_cols = {'Year 1/': 'YEAR'}
cropland_us.rename(columns=rename_cols, inplace=True)

cropland_us.to_pickle('../data/cropland_us.pkl')
cropland_us

### 2.3 Cropland use for cops in each state

In [ ]:
# Read exdata
cropland_state = pd.read_excel('../exdata/additional/Cropland_used_for_crops_19452012_by_state.xls', index_col=None, skiprows=2)
cropland_state = cropland_state.iloc[4:72]

# Filter empty rows
cropland_state = cropland_state.dropna()

# Fill years
cropland_state.set_index('Regions and States', inplace=True)

# Create columns for all intermediate years
cropland_state.columns = cropland_state.columns.astype(str)
years = [int(col) for col in cropland_state.columns]
for idx, year in enumerate(range(min(years), max(years)+4)):
    if str(year) in cropland_state.columns:
        last = cropland_state.iloc[:, idx]
    if str(year) not in cropland_state.columns:
        cropland_state.insert(idx, year, last)

cropland_state = cropland_state.reset_index()

# Melt df
df_dropped = cropland_state.drop('Regions and States', axis=1)
cropland_state_melted = pd.melt(cropland_state, id_vars='Regions and States', value_vars=df_dropped,
                                                var_name='YEAR', value_name='VALUE')

cropland_state = cropland_state_melted.copy()
cropland_state.to_pickle('../data/cropland_state.pkl')
cropland_state

### 2.4 Ethanol produced US

In [ ]:
# Read exdata
ethanol = pd.read_excel('../exdata/additional/PET_PNP_OXY_A_EPOOXE_YOP_MBBLPD_A.xls', sheet_name = 'Data 1', index_col=None, skiprows=2)

# Adapt colname YEAR
rename_cols = {'Date': 'YEAR'}
ethanol.rename(columns=rename_cols, inplace=True)
ethanol['YEAR'] = pd.to_datetime(ethanol['YEAR'], format='%Y%m%d')
ethanol['YEAR'] = ethanol['YEAR'].dt.strftime('%Y')

# Fill nas
ethanol = ethanol.fillna(ethanol.median())
#ethanol.head()

In [ ]:
# Map per region
df = ethanol.drop('U.S. Oxygenate Plant Production of Fuel Ethanol (Thousand Barrels per Day)', axis=1)
df.columns = df.columns.str.replace(r' \(.+$', '', regex=True)

df = pd.melt(df, id_vars='YEAR', value_vars=df,
                var_name='region', value_name='value')
                                                

region_to_state_map = {'East Coast': ['Connecticut', 'Delaware', 'Florida', 'Georgia', 'Maine', 'Maryland',
                                     'Massachusetts', 'New Hampshire', 'New Jersey', 'New York', 'North Carolina',
                                     'Pennsylvania', 'Rhode Island', 'South Carolina', 'Virginia'],
                      'Midwest': ['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri',
                                  'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin'],
                      'Gulf Coast': ['Alabama', 'Louisiana', 'Mississippi', 'Texas'],
                      'Rocky Mountain': ['Arizona', 'Colorado', 'Idaho', 'Montana', 'Nevada', 'New Mexico',
                                         'Utah', 'Wyoming'],
                      'West Coast': ['Alaska', 'California', 'Oregon', 'Washington']}


# Create a new DataFrame to store the results
per_region_df = pd.DataFrame(columns=['state', 'region', 'ethanol', 'YEAR'])

# Iterate through each row in the original DataFrame
for _, row in df.iterrows():
    # Get the region, value, and year values for the current row
    region = row['region']
    value = row['value']
    year = row['YEAR']
    # Get the corresponding states for the region from the reverse mapping
    states_for_region = region_to_state_map.get(region, [])
    # Iterate through the states and create a new row for each state
    for state in states_for_region:
        # Create a new row with the state, region, value, and year values
        new_row = pd.Series({'state': state, 'region': region, 'ethanol': value, 'YEAR': year})
        # Append the new row to the result DataFrame
        per_region_df = per_region_df.append(new_row, ignore_index=True)

#per_region_df

In [ ]:
# add again value in all us
df_ethanol = ethanol[['YEAR','U.S. Oxygenate Plant Production of Fuel Ethanol (Thousand Barrels per Day)']]

df_ethanol = pd.melt(df_ethanol, id_vars='YEAR',
                                 value_vars='U.S. Oxygenate Plant Production of Fuel Ethanol (Thousand Barrels per Day)',
                                 var_name='all_regions', value_name='U.S. ethanol')

df_ethanol = df_ethanol.drop(['all_regions'], axis=1)

ethanol_merged = pd.merge(per_region_df, df_ethanol, on='YEAR', how='inner')
ethanol = ethanol_merged.copy()

ethanol.to_pickle('../data/ethanol.pkl')
ethanol